In [ ]:
import os
import zipfile
import random
from tqdm import tqdm
import shutil
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from os import getcwd
import cv2
#import keras
print(tf.version.VERSION)
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.test.gpu_device_name()

In [ ]:
TRAINING_DIR ="/kaggle/input/binary-multi-retinopathy/data/multi"

train_dataset = tf.keras.utils.image_dataset_from_directory(TRAINING_DIR, validation_split=0.2, 
                                                            subset="training", seed=42, 
                                                            image_size=(340,510),
                                                            label_mode='categorical',
                                                             batch_size=32)
val_dataset = tf.keras.utils.image_dataset_from_directory(TRAINING_DIR, validation_split=0.2, 
                                                          subset="validation", seed=42, 
                                                          image_size=(340,510),
                                                          label_mode='categorical',
                                                           batch_size=32)

In [ ]:
from tensorflow.keras.applications.densenet import DenseNet169
model = tf.keras.models.Sequential([
#     tf.keras.layers.Input(shape=(340,510, 3)),
# tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
#     tf.keras.layers.BatchNormalization(),
    tf.keras.applications.resnet.ResNet50(include_top=False,input_shape=(340,510, 3),weights='imagenet'),
    
    
    
#     tf.keras.layers.MaxPooling2D(2, 2),
#     tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2, 2),
#     tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2, 2),
#     tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2, 2),
#     tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2, 2),
#     tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
#     tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
#     tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')

])

#RMSprop(lr=0.0005)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

In [ ]:
# class myCallback(tf.keras.callbacks.Callback):
#         def on_epoch_end(self,epoch,logs={}):
#             if(logs.get('val_acc') > 0.95):
#                 print("\n Reached 95% accuracy so cancelling training!")
#                 self.model.stop_training=True
# callbacks=myCallback()
old =0
class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self,epoch,logs={}):
            global old
            if(logs.get('val_acc') > old):
                model.save('voice_model.h5',overwrite=True)
                old = logs.get('val_acc')
saver =myCallback()
history = model.fit(train_dataset,
                              epochs=20,
                              verbose=1,
                              validation_data=val_dataset)#,callbacks=[callbacks])